In [1]:
# Required Packages
#!pip install -q pandas_profiling
import numpy as np
import pandas as pd
import pandas_profiling

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold


import time
from datetime import timedelta

Using TensorFlow backend.


In [2]:
# some configuratin flags and variables
verbose=1 # Use in classifier

# Input files
train_url='https://raw.githubusercontent.com/muranjan/datarepo/master/titanic/train.csv'
test_url='https://raw.githubusercontent.com/muranjan/datarepo/master/titanic/test.csv'

# defeine random seed for reproducibility
seed = 123
np.random.seed(seed)

# read training data
train = pd.read_csv(train_url,index_col='PassengerId')

print(train.shape)
print(train.columns)


(891, 11)
Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [3]:
# preview the training dara
#pandas_profiling.ProfileReport(train)
# getting error pandas_profiling like TypeError: concat() got an unexpected keyword argument 'join_axes' 
train[:10]


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S


In [4]:
train.isnull().sum()
#train['Embarked'].value_counts()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [0]:
def FeatureEngineering(df):
    # Drop unwanted features
    df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
    
    # Fill missing data: Age and Fare with the mean, Embarked with most frequent value
    df[['Age']]       = df[['Age']].fillna(value=df[['Age']].mean())
    df[['Fare']]      = df[['Fare']].fillna(value=df[['Fare']].mean())
    df[['Embarked']]  = df[['Embarked']].fillna(value=df['Embarked'].value_counts().idxmax())
    
    # Convert categorical  features into numeric
    df['Sex'] = df['Sex'].map( {'female': 1, 'male': 0} ).astype(int)
      
    # Convert Embarked to one-hot
    enbarked_one_hot = pd.get_dummies(df['Embarked'], prefix='Embarked')
    df = df.drop('Embarked', axis=1)
    df = df.join(enbarked_one_hot)

    return df

In [6]:
train = FeatureEngineering(train)
train.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [0]:
# Let's create X & Y from the train data where Y will have target variable i.e Survived
X = train.drop(['Survived'], axis=1).values.astype(float)
Y = train['Survived'].values
# It is almost always a good idea to perform some scaling of input values when using neural network models (jb).

scale = StandardScaler()
X = scale.fit_transform(X)



In [0]:
def create_model(optimizer='adam', init='uniform'):
    # create model
    if verbose: print("**Create model with optimizer: %s; init: %s" % (optimizer, init) )
    model = Sequential()
    model.add(Dense(16, input_dim=X.shape[1], kernel_initializer=init, activation='relu'))
    model.add(Dense(8, kernel_initializer=init, activation='relu'))
    model.add(Dense(4, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [0]:
run_gridsearch = False

if run_gridsearch:
    
    start_time = time.time()
    if verbose: print (time.strftime( "%H:%M:%S " + "GridSearch started ... " ) )
    optimizers = ['rmsprop', 'adam']
    inits = ['glorot_uniform', 'normal', 'uniform']
    epochs = [50, 100, 200]
    batches = [5, 10]
    
    model = KerasClassifier(build_fn=create_model, verbose=0)
    
    param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=inits)
    grid = GridSearchCV(estimator=model, param_grid=param_grid)
    grid_result = grid.fit(X, Y)
    
    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    if verbose: 
        for mean, stdev, param in zip(means, stds, params):
            print("%f (%f) with: %r" % (mean, stdev, param))
        elapsed_time = time.time() - start_time  
        print ("Time elapsed: ",timedelta(seconds=elapsed_time))
        
    best_epochs = grid_result.best_params_['epochs']
    best_batch_size = grid_result.best_params_['batch_size']
    best_init = grid_result.best_params_['init']
    best_optimizer = grid_result.best_params_['optimizer']
    
else:
    # pre-selected paramters
    best_epochs = 200
    best_batch_size = 5
    best_init = 'glorot_uniform'
    best_optimizer = 'rmsprop'

In [11]:
# Create a classifier with best parameters
model_pred = KerasClassifier(build_fn=create_model, optimizer=best_optimizer, init=best_init, epochs=best_epochs, batch_size=best_batch_size, verbose=verbose)
model_pred.fit(X, Y)

# Read test data
test_df = pd.read_csv(test_url,index_col='PassengerId')
# Prep and clean data
test_df = FeatureEngineering(test_df)
# Create X_test
X_test = test_df.values.astype(float)
# Scaling
X_test = scale.transform(X_test)

# Predict 'Survived'
prediction = model_pred.predict(X_test)

**Create model with optimizer: rmsprop; init: glorot_uniform
Epoch 1/200
891/891 [==============================] - 1s 594us/step - loss: 0.6640 - accuracy: 0.6510
Epoch 2/200
891/891 [==============================] - 0s 479us/step - loss: 0.6068 - accuracy: 0.7845
Epoch 3/200
891/891 [==============================] - 0s 489us/step - loss: 0.5686 - accuracy: 0.8036
Epoch 4/200
891/891 [==============================] - 0s 484us/step - loss: 0.5385 - accuracy: 0.8058
Epoch 5/200
891/891 [==============================] - 0s 534us/step - loss: 0.5151 - accuracy: 0.8148
Epoch 6/200
891/891 [==============================] - 0s 483us/step - loss: 0.4949 - accuracy: 0.8283
Epoch 7/200
891/891 [==============================] - 0s 478us/step - loss: 0.4780 - accuracy: 0.8215
Epoch 8/200
891/891 [==============================] - 0s 516us/step - loss: 0.4589 - accuracy: 0.8283
Epoch 9/200
891/891 [==============================] - 0s 538us/step - loss: 0.4397 - accuracy: 0.8294
Epoch 10/200

In [0]:
submission = pd.DataFrame({
    'PassengerId': test_df.index,
    'Survived': prediction[:,0],
})

submission.sort_values('PassengerId', inplace=True)    
submission.to_csv('Titanic-test_results.csv', index=False)

In [13]:
ls /

bin/                                       etc/    opt/    sys/
boot/                                      home/   proc/   tensorflow-1.15.2/
content/                                   lib/    root/   tmp/
datalab/                                   lib32/  run/    tools/
dev/                                       lib64/  sbin/   usr/
dlib-19.18.0-cp27-cp27mu-linux_x86_64.whl  media/  srv/    var/
dlib-19.18.0-cp36-cp36m-linux_x86_64.whl   mnt/    swift/


In [14]:
ls content/

ls: cannot access 'content/': No such file or directory
